### Imports

In [1]:
# !pip install webrtcvad
# !pip install librosa
# !pip install python_speech_features
# !pip install hmmlearn
import webrtcvad
import soundfile as sf
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa
from sklearn.mixture import GaussianMixture
import scipy.io.wavfile as wav
import sklearn.mixture as mix
import hmmlearn.hmm as gmm
from sklearn.model_selection import train_test_split
from python_speech_features import mfcc, delta
from sklearn.preprocessing import StandardScaler

### Data preProcessing

In [3]:
path_train ="SpeechCommand"
Class_mapping = {'right': 0, 'eight': 1, 'cat': 2, 'tree': 3, 'bed': 4, 'happy': 5, 'go': 6, 'dog': 7, 'no': 8, 'wow':
9, 'nine': 10, 'left': 11, 'stop': 12, 'three': 13, 'sheila': 14, 'one': 15, 'bird': 16, 'zero': 17, 'seven': 18, 'up': 19,
'marvin': 20, 'two': 21, 'house': 22, 'down': 23, 'six': 24, 'yes': 25, 'on': 26, 'five': 27, 'off': 28, 'four': 29,'_background_noise_':30}
new_sample_rate = 8000
sz=0
map=dict()
def log_specgram(audio, sample_rate):
    nperseg = int(round(20 * sample_rate / 1e3))
    noverlap = int(round(10 * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,fs=sample_rate,nperseg=nperseg,noverlap=noverlap,detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + 1e-10)

In [4]:
for word in os.listdir(path_train):
  t=[]
  for prononceiatin in os.listdir(path_train+"/"+word):
      if(prononceiatin[-11:]!=".Identifier" and prononceiatin[-3:]!=".md"):
        samp_rate,samples=wavfile.read(path_train+"/"+word+"/"+prononceiatin)
        samples=signal.resample(samples, int(new_sample_rate/samp_rate * samples.shape[0]))
        if(samples.shape[0]>8000): 
          continue
        sz=max(sz,samples.shape[0])
        t.append(samples)
  if(len(t)!=0):
    map[word]=t

/home/animesh21131/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """


In [6]:
def ratn(s):
    mean = np.mean(s, axis=0)
    std = np.std(s, axis=0)
    s = (s - mean) / std
    return s

In [58]:
specs=dict()
feature=dict()
for i in map:
        t=[]
        ti=[]
        for k in map[i]:
                if(k.shape[0]<sz):
                        l=sz-k.shape[0]
                        k=np.concatenate((np.zeros(l//2),k,np.zeros((l+1)//2)))
                s = librosa.feature.melspectrogram(y=k, sr=new_sample_rate, n_mels=128)
                log_s = librosa.power_to_db(k, ref=np.max)
                mfcc=librosa.feature.mfcc(y=k, n_mfcc=13)
                mfcc=ratn(mfcc)
                t.append(np.array(mfcc))
                freqs, times, spectrogram = log_specgram(k, new_sample_rate)
                spectrogram=ratn(spectrogram)
                ti.append(np.array(spectrogram))
        # flattened_mfcc = np.array([sample.flatten() for sample in t])
        # print(flattened_mfcc.shape)
        feature[i] = np.concatenate(t, axis=0)
        specs[i] = np.concatenate(ti, axis=0)
        # print(feature[i].shape)
        # flattened_specs = np.array([sample.flatten() for sample in t])
        # specs[i] = np.concatenate(specs, axis=0)
        # specs[i]=np.stack(ti)
        print(i)
        # feature[i]=np.stack(t) 

off
up
zero
bird
right
dog
three
on
house
wow
marvin
six
left
one
eight
five
happy
nine
yes
go
sheila
stop
four
down
tree
no
two
bed
cat
seven


### Old Model

In [44]:
f=[]
l=[]
for i in map:
    for j in feature[i]:
        f.append(np.mean(j,axis=1))
        l.append(Class_mapping[i])

In [46]:
# Split data for train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(f), np.array(l), test_size=0.05, random_state=48)


In [47]:
# modeling 

gmm=GaussianMixture(n_components=30,n_init=10,max_iter=200)
gmm.fit(X_train,y_train)

GaussianMixture(max_iter=200, n_components=30, n_init=10)

In [48]:
# calculating accuracy using test data
predictions=gmm.predict(X_test)
score=0
for i in range(0,len(predictions)):
    if predictions[i]==y_test[i]:
        score+=1
    else:
        print("MISMATCH -> Original:",y_test[i],"Predicted:",predictions[i])
print("Accuracy:",score*100/len(predictions),"%")

MISMATCH -> Original: 24 Predicted: 27
MISMATCH -> Original: 8 Predicted: 10
MISMATCH -> Original: 9 Predicted: 21
MISMATCH -> Original: 9 Predicted: 29
MISMATCH -> Original: 21 Predicted: 15
MISMATCH -> Original: 25 Predicted: 7
MISMATCH -> Original: 3 Predicted: 17
MISMATCH -> Original: 3 Predicted: 18
MISMATCH -> Original: 4 Predicted: 3
MISMATCH -> Original: 23 Predicted: 15
MISMATCH -> Original: 22 Predicted: 7
MISMATCH -> Original: 24 Predicted: 16
MISMATCH -> Original: 11 Predicted: 1
MISMATCH -> Original: 27 Predicted: 7
MISMATCH -> Original: 8 Predicted: 21
MISMATCH -> Original: 25 Predicted: 7
MISMATCH -> Original: 28 Predicted: 14
MISMATCH -> Original: 1 Predicted: 2
MISMATCH -> Original: 27 Predicted: 15
MISMATCH -> Original: 3 Predicted: 27
MISMATCH -> Original: 29 Predicted: 1
MISMATCH -> Original: 20 Predicted: 5
MISMATCH -> Original: 0 Predicted: 6
MISMATCH -> Original: 0 Predicted: 1
MISMATCH -> Original: 23 Predicted: 4
MISMATCH -> Original: 1 Predicted: 19
MISMATCH -

In [49]:
# save the model
np.save('model/gmm_weights', gmm.weights_, allow_pickle=False)
np.save('model/gmm_means', gmm.means_, allow_pickle=False)
np.save('model/gmm_covariances', gmm.covariances_, allow_pickle=False)

In [50]:
def predict_gmm_test(audio_files,gmm_model):
    test_feature=[]
    for file in audio_files:
        data, samp_rate=librosa.load(file)
        test_feature.append(np.mean(librosa.feature.mfcc(y=data, sr=samp_rate, n_mfcc=13),axis=1))
    predictions=gmm_model.predict(np.array(test_feature))
    return predictions

In [51]:
# Testing the model for given test.csv
path_train = "SpeechCommandTest"
test_csv=pd.read_csv("test.csv")
result=pd.DataFrame()
result['ID']=test_csv['ID']
test_files=[]
for i in range(len(test_csv)):
  test_files.append(path_train+'/'+test_csv['AUDIO_FILE'][i])
result['Target']=predict_gmm_test(test_files,gmm)
display(result)
result.to_csv("result.csv",index=False)

,ID,Target
0,0,8
1,1,8
2,2,8
3,3,8
4,4,8
...,...,...
6830,6830,8
6831,6831,8
6832,6832,8
6833,6833,8


In [52]:
# load the model
means = np.load('model/gmm_weights.npy')
new_gmm = GaussianMixture(n_components = len(means), covariance_type='full')
new_gmm.means_ = means
new_gmm.weights_ = np.load('model/gmm_weights.npy')
new_gmm.covariances_ = np.load('model/gmm_covariances.npy')
new_gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(new_gmm.covariances_))

### New model

In [65]:
gmms_s=[]
gmms_f=[]
for i in map:
    if(i=='_background_noise_'):
        n_components=4
    else:
        n_components=0
        for _ in i:
            n_components+=1
        if(n_components<4):
            n_components=4
    gmm=GaussianMixture(n_components=n_components,n_init=20,max_iter=600, covariance_type='diag', random_state=42)
    gmm.fit(feature[i])
    gmms_f.append(gmm)
    gmm2=GaussianMixture(n_components=n_components,n_init=20,max_iter=600, covariance_type='diag', random_state=1337)
    gmm2.fit(specs[i])
    gmms_s.append(gmm2)
    print(i)

off
up
zero
bird
right
dog
three
on
house
wow
marvin
six
left
one
eight
five
happy
nine
yes
go
sheila
stop
four
down
tree
no
two
bed
cat
seven


In [84]:
def predict_word(test_file):
    samp_rate,samples=wavfile.read(test_file)
    # if(samples.shape[0]>8000):
    #     print('b',end=' ')
    #     return 30
    samples=signal.resample(samples, int(new_sample_rate/samp_rate * samples.shape[0]))
    if(samples.shape[0]<sz):
        l=sz-samples.shape[0]
        samples=np.concatenate((np.zeros(l//2),samples,np.zeros((l+1)//2)))
    s = librosa.feature.melspectrogram(y=samples, sr=new_sample_rate, n_mels=128)
    log_s = librosa.power_to_db(samples, ref=np.max)
    mfcc=librosa.feature.mfcc(y=samples, n_mfcc=13)
    test_mfcc=ratn(mfcc)
    reqs, times, spectrogram = log_specgram(samples, new_sample_rate)
    test_spec=ratn(spectrogram)
    best_score = float('-inf')
    best_word_idx = -1
    # print(test_mfcc.shape)
    # print(test_spec.shape)
    k=0
    for i in map:
        score = max(gmms_f[k].score(test_mfcc),gmms_s[k].score(test_spec))
        if score > best_score:
            best_score = score
            best_word_idx = Class_mapping[i]
        k+=1
    return best_word_idx

In [66]:
# save the model
k=0
for i in map:
    # os.mkdir('model2/'+i+'/')
    np.save('model2/'+i+'/gmms_f_weights', gmms_f[k].weights_, allow_pickle=False)
    np.save('model2/'+i+'/gmms_f_means', gmms_f[k].means_, allow_pickle=False)
    np.save('model2/'+i+'/gmms_f_covariances', gmms_f[k].covariances_, allow_pickle=False)
    np.save('model2/'+i+'/gmms_s_weights', gmms_s[k].weights_, allow_pickle=False)
    np.save('model2/'+i+'/gmms_s_means', gmms_s[k].means_, allow_pickle=False)
    np.save('model2/'+i+'/gmms_s_covariances', gmms_s[k].covariances_, allow_pickle=False)
    k+=1

In [85]:
# Testing the model for given test.csv
path_train = "SpeechCommandTest"
test_csv=pd.read_csv("test.csv")
result=pd.DataFrame()
result['ID']=test_csv['ID']
test_files=[]
for i in range(len(test_csv)):
  test_files.append(predict_word(path_train+'/'+test_csv['AUDIO_FILE'][i]))
result['Target']=test_files
display(result)
result.to_csv("result_m2.csv",index=False)

,ID,Target
0,0,0
1,1,14
2,2,18
3,3,20
4,4,20
...,...,...
6830,6830,18
6831,6831,14
6832,6832,22
6833,6833,22


In [81]:
# load the model
new_gmms_f=[]
new_gmms_s=[]
for i in map:
    if(i=='_background_noise_'):
        n_components=4
    else:
        n_components=0
        for _ in i:
            n_components+=1
        if(n_components<4):
            n_components=4
    means = np.load('model2/'+i+'/gmms_f_weights.npy')
    gmm=GaussianMixture(n_components=n_components,n_init=20,max_iter=600, covariance_type='diag', random_state=42)
    gmm.means_ = means
    gmm.weights_ = np.load('model2/'+i+'/gmms_f_weights.npy')
    gmm.covariances_ = np.load('model2/'+i+'/gmms_f_covariances.npy')
    # gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(gmm.covariances_))
    new_gmms_f.append(gmm)
    m2 = np.load('model2/'+i+'/gmms_s_weights.npy')
    gmm2=GaussianMixture(n_components=n_components,n_init=20,max_iter=600, covariance_type='diag', random_state=42)
    gmm2.means_ = m2
    gmm2.weights_ = np.load('model2/'+i+'/gmms_s_weights.npy')
    gmm2.covariances_ = np.load('model2/'+i+'/gmms_s_covariances.npy')
    # gmm2.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(gmm2.covariances_))
    new_gmms_f.append(gmm2)

### Final Model

In [5]:
frame_duration = 25 #ms
frame_size = int(new_sample_rate * (frame_duration / 1000.0))
vad = webrtcvad.Vad()
vad.set_mode(3)
def extract_speech_segments(audio_path):
    audio, sr = librosa.load(audio_path, sr = new_sample_rate)
    speech_segments = list()
    start , end = -1 , 0
    for i in range(0, len(audio), frame_size):
        frame = audio[i:i + frame_size]
        frame = np.int16(frame * 32768)
        if vad.is_speech(frame.tobytes(), sample_rate=sr):
            if start == -1:
                start = i
            end = i + frame_size
        else:
            if start != -1:
                speech_segments.append((start, end))
                start = -1
    if start != -1:
        speech_segments.append((start, end))
    return speech_segments

In [6]:
from python_speech_features import mfcc, delta
def extract_features(wav_file):
    samp_rate, samples = wav.read(wav_file)
    samples=signal.resample(samples, int(new_sample_rate/samp_rate * samples.shape[0]))
    mfcc_feat = mfcc(samples, new_sample_rate)
    dfeat = delta(mfcc_feat, 2)
    features = np.concatenate((mfcc_feat, dfeat, delta(dfeat,2)), axis=1)
    scaler = StandardScaler()
    scaler.fit(features)
    normalized_features = scaler.transform(features) #
    return normalized_features

In [7]:
from sklearn.model_selection import train_test_split
def train_model(label):
    wav_files = os.listdir(os.path.join(path_train, label))
    train_files, test_files = train_test_split(wav_files, test_size=0.2, random_state=42)
    train_features = np.concatenate([ extract_features(os.path.join(path_train, label, file)) for file in train_files ])
    model = gmm.GMMHMM(n_components=6, n_mix=3, covariance_type='diag', n_iter=10, random_state=42)
    model.fit(train_features)
    return model, test_files

In [8]:
import pickle
models = {}
for i in map:
    model, test_files = train_model(i)
    models[i] = (model, test_files)
    pickle.dump(model, open(f'model3/{i}.pkl', 'wb'))

In [11]:
i='_background_noise_'
model, test_files = train_model(i)
models[i] = (model, test_files)
pickle.dump(model, open(f'model3/{i}.pkl', 'wb'))
map['_background_noise_']=list("gg")

/home/animesh21131/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/animesh21131/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/animesh21131/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
def pred(file_path):
    try:
        features = extract_features(file_path)
    except:
        return '_background_noise_'
    best_score = models['_background_noise_'][0].score(features)
    best_label = '_background_noise_'
    for i in map:
        score = models[i][0].score(features)
        if score > best_score:
            best_score = score
            best_label = i
    return best_label

In [14]:
tc=0
tf=0
b=0
for i in map:
    c=0
    k=0
    for j in models[i][1]:
        lab=pred(os.path.join(path_train, i, j))
        if lab==i:
            c+=1
        if(lab=='_background_noise_'):
            b+=1
        k+=1
    print(f'Accuracy for {i}: {c / k:.2f}')
    tc+=c
    tf+=k
print(f'Overall accuracy: {tc / tf:.2f}')
print(b)

Accuracy for off: 0.76
Accuracy for up: 0.85
Accuracy for zero: 0.89
Accuracy for bird: 0.88
Accuracy for right: 0.88
Accuracy for dog: 0.87
Accuracy for three: 0.80
Accuracy for on: 0.83
Accuracy for house: 0.89
Accuracy for wow: 0.94
Accuracy for marvin: 0.94
Accuracy for six: 0.92
Accuracy for left: 0.83
Accuracy for one: 0.91
Accuracy for eight: 0.94
Accuracy for five: 0.84
Accuracy for happy: 0.94
Accuracy for nine: 0.92
Accuracy for yes: 0.92
Accuracy for go: 0.83
Accuracy for sheila: 0.94
Accuracy for stop: 0.85
Accuracy for four: 0.90
Accuracy for down: 0.84
Accuracy for tree: 0.90
Accuracy for no: 0.76
Accuracy for two: 0.83
Accuracy for bed: 0.93
Accuracy for cat: 0.92
Accuracy for seven: 0.83


/home/animesh21131/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy for _background_noise_: 1.00
Overall accuracy: 0.87
122


In [15]:
# Testing the model for given test.csv
path_train = "SpeechCommandTest"
test_csv=pd.read_csv("test.csv")
result=pd.DataFrame()
result['ID']=test_csv['ID']
test_files=[]
for i in range(len(test_csv)):
  test_files.append(Class_mapping[pred(path_train+'/'+test_csv['AUDIO_FILE'][i])])
result['Target']=test_files
display(result)
result.to_csv("result_m3.csv",index=False)

,ID,Target
0,0,24
1,1,26
2,2,23
3,3,6
4,4,28
...,...,...
6830,6830,19
6831,6831,16
6832,6832,11
6833,6833,19


In [16]:
def find_and_load_model(model_name, directory):
    flag=True
    for file in os.listdir(directory):
        if file == model_name:
            flag=False
            break
    if flag:
        raise FileNotFoundError(f"No model file named '{model_name}' found in the directory.")
    loaded_model = pickle.load(open(os.path.join(directory, file), 'rb'))
    return loaded_model
for label in map:
    loaded_model = find_and_load_model(f'{label}.pkl', 'model3/')

Significance of models: <br>
If we observe first model uses simple single gmm srategy to solve this problem but it fails prety badly <br><br>
But the second model uses distributed gmms per word. it utilizes concatenated mfcc, resampling and concatenated spectogram. <br>
It also utillizes normalizing and logrithmic approaches. <br>
It does well as compared to first model but not to desired extent. <br>
<br>
In third model we try incoperating slopes of mfcc anfd use VAD and the results amaze us. Much better accuracy <br> <br>
ML is all about experiments folks, You can further make this better by tunning parameters of the models and incoperating more of the temporal information or probably by utilizing Deep Learning Frameworks to analyze log_spectogram and mfcc features together (with help of CNN's ig) <br><br>
Happy Codding!!